# Coding custom decision tree algorithm (modified CART)
## Part 1: Decision trees overview


### Introduction

This is the first in a series of articles that cover an implementation of a **custom decision tree algorithm**.

Decision trees are a very interesting class of machine learning models. 
Putting various caveats aside for the moment, decision trees are particularly interesting because of their interpretability, simplicity in communicating their decision logic, and the insight they give into predictivity of variables.
Trees can be used on their own, or they can be bundled together in ensembles to form boosted trees, random forests, etc. to become even more powerful predictors. 
There are a couple of standard algorithms implementing the basic decision tree logic ([see Wikipedia](https://en.wikipedia.org/wiki/Decision_tree_learning)).

Now, scikit-learn provides Python coders with a great implementation of a modified CART algorithm. It has most of the things you might need from your decision tree and it is also quite fast - so why code your own tree?


### Why write a custom decision tree?

There may be plenty of reasons to do it (optimization/improvement/extension of whatever, scaling, more control over something, etc.), but I decided to do it at some point when I started exploring online learning. Online learning is the ability of a machine learning model to learn continuously on a stream of data, improving with each new record, and not having to go through the whole dataset in order to learn.
Traditional decision trees, and CART implementation in particular, are not able to do that.

A CART tree takes a greedy, global, approach such that it scans through the whole dataset for the best choice of variable and the value of the variable on which to split the dataset and then repeats the process on child nodes until it builds the tree. (If this sentence makes no sense to you - there's a simple review of the CART algo below, ignore this for now.)

So to finally answer the question in the title of the section, while reading about online learning I've stumbled upon **Hoeffding trees**. This is an implementation of the decision tree logic which can learn online, on data streams, and which was proved to asymptotically converge to the standard trees in the original paper which proposed the idea. (Find the article [here](https://homes.cs.washington.edu/~pedrod/papers/kdd00.pdf), I highly recommend it.)
Since Hoeffding trees are not part of the scikit-learn, I wanted to code them myself. As a step in that direction, I thought it would be fun to start from the standard CART algorithm.

### What to expect from this series

In all honesty, **having coded neural networks and various types of regressions before, I was expecting the code to be very simple** - I definitely expected to write less than 100 lines for a basic implementation.

**I was very wrong.**

I started coding bottom-up, quite a bad idea in retrospect, but quite reasonable considering that I expected it to be a very simple problem. In the end it took me a few hours a day for a few days to complete the code and write this up. (I did this after-work which may explain why I was slow.)

The reason I decided to put it up online is because I think that it's quite cool. Even though it's not as fast or as versatile as the scikit-learn version, you have the advantage that it's all there in front of you, quite simple and transparent. I will also give some suggestions for optimization which I believe could bring the speed up to the scikit-learn level or beyond. The algo is also not always 100% straightforward and you can solve some problems in multiple ways - so I think it makes for an interesting discussion.

To conclude, I want to emphasize that this is definitely not the best possible implementation. I thought about going through the algorithm once again, top-bottom, in order to optimize it fully. In the end I decided not to do so, because that was not the point anyway - **the point was to get a deeper, low-level understanding of the inner workings of decision trees, and to have a simple, easily understandable code**. If I spend 10 more hours on optimization, it'll run a bit faster, but it will be much more complex conceptually.

### A review of the CART algorithm

I tried briefly to find a good pseudocode for the CART algorithm.
First I looked at articles such as [this one](https://machinelearningmastery.com/classification-and-regression-trees-for-machine-learning/) (I highly recommend this website in general, by the way) which are pretty good at giving you general ideas and broad overview of how the decision trees work, but no details about inner workings.
Next, I opened up "The elements of statistical learning" by Hastie et.al. (chapter 9 talks about the trees and explains CART), but again, there was no good pseudocode that I could easily and quickly turn into Python code.
So long-story-short **I abandoned this search quickly after finding some high-level descriptions of what the code is supposed to do, and I worked from there** (ignoring the wise saying that "one month in a lab can save you a couple of hours in a library").

What I learned online is what I knew already, and it is more-or-less that the CART algorithm works as follows. 
Suppose you have some dataset which consists of a number of features $(x_1, x_2, ..., x_n)$, and a target variable $y$. Let's say you want to predict whether an animal is a dog or a duck. $y$ will be labels, "dog" and "duck", on your train set, and $\{x_i\}$ will be various predictive variables: number of legs, indicator whether the animal has wings, etc. Now, your dataset has features and target organized in columns, and it has records organized in rows (aka dataframe).

**CART algorithm** will scan through the whole dataset looking for the variable that best splits the records into two lists - one having as many ducks and the other having as many dogs as possible, with each list having as few of the other records as possible. (The concept of "impurity" will be introduced in Part 2 to better quantify this.)

In order to find the best **splitting feature**, the feature on which to split the "parent" dataset into two "child" datatsets, the algorithm has to go through all values of all features and see how good the resulting child datasets would be (in terms of impurity). For this to work, it has to *see* all records - thus being incompatible with online learning.

Once the algo produces the splitting condition (choice of feature and threshold for which to split), records are compared to the condition and split in two. The algo continues iteratively on each of the child nodes, then on each of their child nodes, and so on until it reaches the stopping condition.

There are three main types of **stopping conditions** - all of them implemented to prevent model from overfitting the data.
The first condition is a prespecified maximum depth of the tree. For example we can impose that the tree may not grow beyond second level, so that the root (level 0), has two child nodes (level 1), and each of those has two child nodes (level 2). This prevents the tree from growing too much overall.
The second stopping condition is the size of the dataset that enters the node. We could say that we do not want to split any node during training which has less than 100 records in it. This may prevent some branches from reaching the maximum depth.
Finally, the last condition is the impurity decrease. For now, let's say this is a condition that says not to split the node if you can't really gain much by splitting it. This, again, only prevents some branches from reaching the maximum depth.

### Conclusion

I've described why I decided to code my own custom decision tree model and I explained that it was fun and informative enough that I decided to share it. 
I've also given you a broad, high-level overview of the decision trees and the CART algorithm. 

I hope you find this series interesting and informative. I will be covering some probability theory, some simple coding, a touch of machine learning, some general problem solving and a pinch of data structures (binary trees).

In the next part, part 2, is where we will start building the decision tree bottom-up. We will begin with the function that computes Gini impurity index of a list of numbers (in practice a list of values of the target variable).